In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools

# Read the CSV file
vt24 = pd.read_csv("C:/Users/bwdea/OneDrive/Documents/Basketball/Virginia Tech/Lineups/Games/VT-All Games.csv")

# Your roster in the order you want with minutes
roster_all = [
    'Tobi Lawal',
    'Jaden Schutt',
    'Tyler Johnson',
    'Jaydon Young',
    'Brandon Rechsteiner',
    'Ben Burnham',
    'Mylyjael Poteat',
    'Ben Hammond',
    'Rodney Brown Jr.',
    'Patrick Wessler',
    'Connor Serven',
    'Ryan Jones Jr.',
    'Connor Venable',
    'Peter Carr'
]
minutes_all = [791, 628, 794, 782, 840, 788, 616, 845, 611, 646, 213, 51, 10, 4]

# Define player positions (primary -> secondary -> tertiary, left to right)
player_positions = {
    'Tobi Lawal': [4, 5],
    'Jaden Schutt': [2, 3],
    'Tyler Johnson': [3, 4],
    'Jaydon Young': [2, 3],
    'Brandon Rechsteiner': [1, 2],
    'Ben Burnham': [4],
    'Mylyjael Poteat': [5],
    'Ben Hammond': [1, 2],
    'Rodney Brown Jr.': [3, 2],
    'Patrick Wessler': [5],
    'Connor Serven': [4, 3, 5],
    'Ryan Jones Jr.': [4, 5],
    'Connor Venable': [2, 3],
    'Peter Carr': [3]
}

# Get all player columns
player_columns = ['Player1', 'Player2', 'Player3', 'Player4', 'Player5']

# Calculate plus/minus for each player
player_stats = {}
for _, row in vt24.iterrows():
    plus_minus = row['+/-']
    
    for col in player_columns:
        player_name = row[col]
        if player_name not in player_stats:
            player_stats[player_name] = 0
        player_stats[player_name] += plus_minus

# Apply name replacements
player_stats['Tobi Lawal'] = player_stats.pop('Toibu Tobi Lawal', 0)
player_stats['Rodney Brown Jr.'] = player_stats.pop('Rodney Brown', 0)

# Filter to only players with at least 40 minutes
roster = []
minutes = []
num_games = 32

for player, mins in zip(roster_all, minutes_all):
    if mins >= 40:
        roster.append(player)
        minutes.append(mins)

# Calculate plus/minus per minute for filtered players
plusminuses = {player: player_stats.get(player, 0) / mins 
               for player, mins in zip(roster, minutes)}

# Calculate bounds based on season average
bounds = {}
for player, mins in zip(roster, minutes):
    avg_mpg = mins / num_games
    lower = max(0, avg_mpg - 5)
    upper = avg_mpg + 5
    bounds[player] = (lower, upper)

# Sort players by plus/minus per minute (best to worst)
ranked_players = sorted(roster, key=lambda p: plusminuses[p], reverse=True)

print("Player Rankings by +/- per minute:")
print("=" * 70)
for i, player in enumerate(ranked_players, 1):
    print(f"{i:2}. {player:20} {plusminuses[player]:+.4f} per min  "
          f"({bounds[player][0]:.1f}-{bounds[player][1]:.1f} min range)")
    positions_str = ", ".join(str(p) for p in player_positions.get(player, []))
    print(f"     Can play positions: {positions_str}")
print()

# Calculate target minutes for each player (midpoint of their bounds)
target_minutes = {}
for player in roster:
    target_minutes[player] = (bounds[player][0] + bounds[player][1]) / 2

print("Target Minutes:")
print("=" * 70)
# Sort by target minutes, highest to lowest
sorted_by_minutes = sorted(ranked_players, key=lambda p: target_minutes[p], reverse=True)
for player in sorted_by_minutes:
    print(f"{player:20}: {target_minutes[player]:.1f} min")
print()

# Generate sequential lineup assignments
print("=" * 70)
print("SEQUENTIAL LINEUP ASSIGNMENTS:")
print("=" * 70)

# Function to check if a 5-player combination covers all positions
def covers_all_positions(players):
    covered = set()
    for player in players:
        covered.update(player_positions.get(player, []))
    return len(covered) >= 5 and all(pos in covered for pos in range(1, 6))

# Function to assign positions to a lineup
def assign_positions_to_lineup(lineup):
    """Smart position assignment - prioritizes rarest positions first"""
    lineup_pos_assignment = {}
    
    # Count how many players can play each position
    position_availability = {}
    for position in range(1, 6):
        available_players = [p for p in lineup if position in player_positions.get(p, [])]
        position_availability[position] = len(available_players)
    
    # Sort positions by availability (rarest first)
    positions_by_rarity = sorted(position_availability.keys(), key=lambda p: position_availability[p])
    
    # Assign rarest positions first
    for position in positions_by_rarity:
        for player in lineup:
            if player in lineup_pos_assignment:
                continue
            if position in player_positions.get(player, []):
                lineup_pos_assignment[player] = position
                break
    
    return lineup_pos_assignment

# Track remaining minutes for each player
remaining_minutes = target_minutes.copy()

lineup_schedule = []
total_minutes_assigned = 0

iteration = 0
while total_minutes_assigned < 40 and iteration < 50:  # Safety limit
    iteration += 1
    
    # Get active players (those with remaining minutes)
    active_players = [p for p in remaining_minutes if remaining_minutes[p] > 0.01]
    
    if len(active_players) < 5:
        print(f"\n⚠ Only {len(active_players)} players remaining with minutes")
        print(f"   Active players: {', '.join(active_players)}")
        print(f"   Cannot form a complete lineup")
        break
    
    # Generate all possible 5-player combinations from active players
    all_combinations = list(itertools.combinations(active_players, 5))
    
    # Find valid lineups that cover all positions
    valid_lineups = []
    for combo in all_combinations:
        if covers_all_positions(combo):
            # Calculate lineup plus/minus (simple sum)
            lineup_rating = sum(plusminuses[p] for p in combo)
            valid_lineups.append((combo, lineup_rating))
    
    if not valid_lineups:
        print(f"\n⚠ No valid lineups found with remaining {len(active_players)} players")
        print(f"   Active players and their positions:")
        for p in active_players:
            pos_str = ", ".join(str(pos) for pos in player_positions.get(p, []))
            print(f"     {p}: positions {pos_str}, {remaining_minutes[p]:.1f} min remaining")
        break
    
    # Sort by rating and pick the best
    valid_lineups.sort(key=lambda x: x[1], reverse=True)
    best_lineup = valid_lineups[0][0]
    best_rating = valid_lineups[0][1]
    
    # Find minimum minutes among these 5 players
    min_minutes = min(remaining_minutes[p] for p in best_lineup)
    
    # Don't exceed 40 total minutes
    minutes_remaining_in_game = 40 - total_minutes_assigned
    lineup_duration = min(min_minutes, minutes_remaining_in_game)
    
    # Assign positions
    position_assignment = assign_positions_to_lineup(best_lineup)
    
    # Record this lineup
    lineup_schedule.append({
        'lineup': best_lineup,
        'duration': lineup_duration,
        'rating': best_rating,
        'positions': position_assignment
    })
    
    # Update remaining minutes for each player in the lineup
    for player in best_lineup:
        remaining_minutes[player] -= lineup_duration
    
    total_minutes_assigned += lineup_duration
    
    # If we've hit exactly 40 minutes, we're done
    if total_minutes_assigned >= 39.99:  # Small epsilon for floating point
        break

# Sort lineups by duration (longest first)
lineup_schedule_sorted = sorted(lineup_schedule, key=lambda x: x['duration'], reverse=True)

# Print the lineup schedule
print(f"\nGenerated {len(lineup_schedule_sorted)} lineup rotation(s) covering {total_minutes_assigned:.1f} minutes")
print("(Sorted by duration - longest stints first)\n")

for i, lineup_info in enumerate(lineup_schedule_sorted, 1):
    print(f"Lineup {i}: {lineup_info['duration']:.1f} minutes (Rating: {lineup_info['rating']:+.4f} per min)")
    print("-" * 70)
    
    for player in lineup_info['lineup']:
        pos = lineup_info['positions'].get(player, "?")
        print(f"  Position {pos}: {player:20} (+/- per min: {plusminuses[player]:+.4f})")
    
    expected_plus_minus = lineup_info['duration'] * lineup_info['rating']
    print(f"  Expected +/- for this stint: {expected_plus_minus:+.2f}")
    print()

# Show summary
print("=" * 70)
print("ROTATION SUMMARY:")
print("=" * 70)

player_lineup_minutes = defaultdict(float)
player_position_minutes = defaultdict(lambda: defaultdict(float))

for lineup_info in lineup_schedule:
    for player in lineup_info['lineup']:
        player_lineup_minutes[player] += lineup_info['duration']
        pos = lineup_info['positions'].get(player)
        if pos:
            player_position_minutes[player][pos] += lineup_info['duration']

print("\nMinutes by player in rotation:")
for player in sorted(player_lineup_minutes.keys(), key=lambda p: player_lineup_minutes[p], reverse=True):
    target = target_minutes[player]
    scheduled = player_lineup_minutes[player]
    diff = scheduled - target
    print(f"  {player:20}: {scheduled:5.1f} min scheduled (target: {target:5.1f}, diff: {diff:+5.1f})")

print("\nPosition breakdown by player:")
for player in sorted(player_lineup_minutes.keys(), key=lambda p: player_lineup_minutes[p], reverse=True):
    if player in player_position_minutes:
        positions_str = ", ".join(f"Pos {pos}: {mins:.1f}m" for pos, mins in sorted(player_position_minutes[player].items()))
        print(f"  {player:20}: {positions_str}")

# Verify position coverage
print("\nTotal minutes by position:")
position_totals = defaultdict(float)
for lineup_info in lineup_schedule:
    for player, pos in lineup_info['positions'].items():
        position_totals[pos] += lineup_info['duration']

for pos in range(1, 6):
    print(f"  Position {pos}: {position_totals[pos]:.1f}/40 minutes")

total_expected_plus_minus = sum(l['duration'] * l['rating'] for l in lineup_schedule)

# Calculate actual team plus/minus from the data
actual_team_plus_minus = sum(player_stats.values())

# Scale optimized +/- to full season
total_expected_plus_minus_season = total_expected_plus_minus * 32

print(f"\nTotal Expected Team +/- (Per Game):   {total_expected_plus_minus:+.2f}")
print(f"Total Expected Team +/- (Season):     {total_expected_plus_minus_season:+.2f}")
print(f"Actual Team +/- (Season):             {actual_team_plus_minus:+.2f}")
print(f"Improvement over season:              {total_expected_plus_minus_season - actual_team_plus_minus:+.2f}")
print(f"Improvement per game:                 {(total_expected_plus_minus_season - actual_team_plus_minus) / 32:+.2f}")

# Check if we covered all 40 minutes
if total_minutes_assigned < 40:
    print(f"\n⚠ WARNING: Only covered {total_minutes_assigned:.1f} of 40 minutes")
    print(f"   Remaining players:")
    for p in remaining_minutes:
        if remaining_minutes[p] > 0.01:
            print(f"     {p}: {remaining_minutes[p]:.1f} min")

# Load game results and analyze impact
print("\n" + "=" * 70)
print("GAME-BY-GAME IMPACT ANALYSIS:")
print("=" * 70)

try:
    game_results = pd.read_csv("C:/Users/bwdea/OneDrive/Documents/Basketball/Research/Game Results 2024-25.csv")
    
    print("\nOptimizing each game individually (accounting for injuries)...\n")
    
    game_optimizations = []
    games_changed = 0
    losses_to_wins = []
    wins_to_losses = []
    closer_losses = []
    bigger_wins = []
    
    for _, game in game_results.iterrows():
        game_id = game['id']
        opponent = game['Opponent']
        actual_margin = game['Margin']
        actual_result = game['Result']
        out_players_str = game['Out']
        
        # Parse out players
        out_players = []
        if pd.notna(out_players_str) and out_players_str.strip():
            out_players = [p.strip() for p in out_players_str.split(',')]
        
        # Get available players for this game
        available_players = [p for p in roster if p not in out_players]
        
        if len(available_players) < 5:
            print(f"⚠ Game {game_id}: Not enough players available")
            continue
        
        # Calculate target minutes for available players
        game_target_minutes = {}
        for player in available_players:
            game_target_minutes[player] = target_minutes[player]
        
        # Run optimization for this specific game
        game_remaining_minutes = game_target_minutes.copy()
        game_lineup_schedule = []
        game_total_minutes = 0
        
        iteration = 0
        while game_total_minutes < 40 and iteration < 50:
            iteration += 1
            
            active = [p for p in game_remaining_minutes if game_remaining_minutes[p] > 0.01]
            
            if len(active) < 5:
                break
            
            # Find valid lineups from available players
            valid = []
            for combo in itertools.combinations(active, 5):
                if covers_all_positions(combo):
                    rating = sum(plusminuses[p] for p in combo)
                    valid.append((combo, rating))
            
            if not valid:
                break
            
            valid.sort(key=lambda x: x[1], reverse=True)
            best_lineup = valid[0][0]
            best_rating = valid[0][1]
            
            min_mins = min(game_remaining_minutes[p] for p in best_lineup)
            duration = min(min_mins, 40 - game_total_minutes)
            
            position_assignment = assign_positions_to_lineup(best_lineup)
            
            game_lineup_schedule.append({
                'lineup': best_lineup,
                'duration': duration,
                'rating': best_rating,
                'positions': position_assignment
            })
            
            for player in best_lineup:
                game_remaining_minutes[player] -= duration
            
            game_total_minutes += duration
            
            if game_total_minutes >= 39.99:
                break
        
        # Calculate expected +/- for this game
        game_expected_plus_minus = sum(l['duration'] * l['rating'] for l in game_lineup_schedule)
        
        # Calculate optimized margin
        optimized_margin = actual_margin + game_expected_plus_minus - (actual_team_plus_minus / 32)
        
        # Determine optimized result
        if optimized_margin > 0:
            optimized_result = 'W'
        elif optimized_margin < 0:
            optimized_result = 'L'
        else:
            optimized_result = 'T'
        
        game_optimizations.append({
            'id': game_id,
            'opponent': opponent,
            'actual_margin': actual_margin,
            'actual_result': actual_result,
            'optimized_margin': optimized_margin,
            'optimized_result': optimized_result,
            'expected_plus_minus': game_expected_plus_minus,
            'out_players': out_players
        })
        
        # Check if result changed
        if actual_result != optimized_result:
            games_changed += 1
            change_str = f"Game {game_id:2}: {opponent:20} "
            change_str += f"Actual: {actual_result} ({actual_margin:+3.0f})  →  "
            change_str += f"Optimized: {optimized_result} ({optimized_margin:+.1f})"
            if out_players:
                change_str += f"  [Out: {', '.join(out_players)}]"
            
            if actual_result == 'L' and optimized_result == 'W':
                losses_to_wins.append(change_str)
            elif actual_result == 'W' and optimized_result == 'L':
                wins_to_losses.append(change_str)
                
            print(change_str)
        else:
            # Track close games that get better/worse
            if actual_result == 'L' and abs(actual_margin) <= 10:
                if optimized_margin > actual_margin:
                    closer_losses.append((game_id, opponent, actual_margin, optimized_margin))
            elif actual_result == 'W' and actual_margin <= 10:
                if optimized_margin > actual_margin:
                    bigger_wins.append((game_id, opponent, actual_margin, optimized_margin))
    
    # Calculate season totals from game-by-game optimization
    total_optimized_plus_minus_season = sum(g['expected_plus_minus'] for g in game_optimizations)
    optimized_per_game = total_optimized_plus_minus_season / len(game_optimizations)
    
    # Summary
    print("\n" + "-" * 70)
    print("SUMMARY:")
    print("-" * 70)
    
    actual_wins = len(game_results[game_results['Result'] == 'W'])
    actual_losses = len(game_results[game_results['Result'] == 'L'])
    
    optimized_wins = actual_wins + len(losses_to_wins) - len(wins_to_losses)
    optimized_losses = actual_losses - len(losses_to_wins) + len(wins_to_losses)
    
    print(f"\nActual Record:                        {actual_wins}-{actual_losses}")
    print(f"Optimized Record (with injuries):     {optimized_wins}-{optimized_losses}")
    print(f"\nGames with changed outcomes: {games_changed}")
    print(f"  Losses → Wins: {len(losses_to_wins)}")
    print(f"  Wins → Losses: {len(wins_to_losses)}")
    
    print(f"\n" + "-" * 70)
    print("SEASON +/- COMPARISON (Game-by-Game Optimization):")
    print("-" * 70)
    print(f"Actual Team +/- (Season):             {actual_team_plus_minus:+.2f}")
    print(f"Actual Team +/- (Per Game):           {actual_team_plus_minus / 32:+.2f}")
    print(f"Optimized Team +/- (Season):          {total_optimized_plus_minus_season:+.2f}")
    print(f"Optimized Team +/- (Per Game):        {optimized_per_game:+.2f}")
    print(f"Improvement over season:              {total_optimized_plus_minus_season - actual_team_plus_minus:+.2f}")
    print(f"Improvement per game:                 {(total_optimized_plus_minus_season - actual_team_plus_minus) / 32:+.2f}")
    
    if closer_losses:
        print(f"\nClose losses that get closer (but still losses):")
        for game_id, opp, actual, opt in closer_losses[:5]:
            print(f"  Game {game_id}: {opp:20} {actual:+3.0f} → {opt:+.0f}")
    
    if bigger_wins:
        print(f"\nClose wins that become more comfortable:")
        for game_id, opp, actual, opt in bigger_wins[:5]:
            print(f"  Game {game_id}: {opp:20} {actual:+3.0f} → {opt:+.0f}")
            
except FileNotFoundError:
    print("\n⚠ Game results file not found")
except Exception as e:
    print(f"\n⚠ Error analyzing game results: {e}")
    import traceback
    traceback.print_exc()

Player Rankings by +/- per minute:
 1. Rodney Brown Jr.     +0.0704 per min  (14.1-24.1 min range)
     Can play positions: 3, 2
 2. Patrick Wessler      +0.0310 per min  (15.2-25.2 min range)
     Can play positions: 5
 3. Connor Serven        +0.0094 per min  (1.7-11.7 min range)
     Can play positions: 4, 3, 5
 4. Ryan Jones Jr.       +0.0000 per min  (0.0-6.6 min range)
     Can play positions: 4, 5
 5. Ben Burnham          -0.0178 per min  (19.6-29.6 min range)
     Can play positions: 4
 6. Brandon Rechsteiner  -0.0179 per min  (21.2-31.2 min range)
     Can play positions: 1, 2
 7. Jaydon Young         -0.0320 per min  (19.4-29.4 min range)
     Can play positions: 2, 3
 8. Tyler Johnson        -0.0416 per min  (19.8-29.8 min range)
     Can play positions: 3, 4
 9. Ben Hammond          -0.0734 per min  (21.4-31.4 min range)
     Can play positions: 1, 2
10. Tobi Lawal           -0.0771 per min  (19.7-29.7 min range)
     Can play positions: 4, 5
11. Mylyjael Poteat      -0.082